# China Eastern Airlines Flight 5735

#### Load Python tools

In [6]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

## Read [data](https://www.flightradar24.com/data/aircraft/b-1791) from Flightradar24

In [9]:
src = pd.read_csv("../data/raw/MU5735_2b367bc.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/MU5735_2b367bc.csv'

In [ ]:
src.columns = src.columns.str.lower()

In [ ]:
src["date"] = pd.to_datetime(src["utc"]).dt.date
src["time"] = pd.to_datetime(src["utc"]).dt.time

In [ ]:
src[["latitude", "longitude"]] = src["position"].str.split(",", expand=True)

In [ ]:
src = src[
    [
        "date",
        "time",
        "timestamp",
        "altitude",
        "speed",
        "direction",
        "latitude",
        "longitude",
    ]
].sort_values("time")

In [ ]:
geo_df = gpd.GeoDataFrame(
    src, geometry=gpd.points_from_xy(src["longitude"], src["latitude"])
)

In [ ]:
geo_df["flight"] = "China Eastern Airlines Flight 5735"

In [ ]:
geo_df["date"] = geo_df["date"].astype(str)

#### There's a 30-minute gap in the flight transponder, so we'll create two lines

In [ ]:
after = geo_df[geo_df["timestamp"] > 1647840425]

In [ ]:
before = geo_df[geo_df["timestamp"] <= 1647840425]

In [ ]:
before_lines = (
    before.groupby(["date", "flight"])["geometry"]
    .apply(lambda x: LineString(x.tolist()) if x.size > 1 else x.tolist())
    .reset_index()
)

In [ ]:
after_lines = (
    after.groupby(["date", "flight"])["geometry"]
    .apply(lambda x: LineString(x.tolist()) if x.size > 1 else x.tolist())
    .reset_index()
)

In [ ]:
before_lines.plot()

## Export 

In [17]:
src.to_csv("../../usc/data/geo/china-eastern-crash-points.csv", index=False)
after_lines.to_file(
    "../../usc/data/geo/china-eastern-crash-lines-after.geojson", driver="GeoJSON"
)
before_lines.to_file(
    "../../usc/data/geo/china-eastern-crash-lines-before.geojson", driver="GeoJSON"
)